Instrument ID, arm, camera, mediapipe set up

In [1]:
import os 
import sys
import cv2
import numpy as np
import math
import pyrealsense2 as rs
import time
import struct
import socket
import torch
import random
# Import modules from voice-control-instrument-id/voice_instrument_functions.py
#instrument_module_path = os.path.abspath(os.path.join(__file__, "..", "..", "voice-control-instrument-id"))
notebook_path = os.getcwd()  # Current working directory
instrument_module_path = os.path.abspath(os.path.join(notebook_path, "..", "voice-control-instrument-id"))
sys.path.append(instrument_module_path) 
from voice_instrument_functions import *
from audio_utils import *

# Load instrument identification model
inst_model = load_model('../voice-control-instrument-id/models/instrument_detector_model.pt', False)

from robot_control_functions import * # Because of mediapipe, need to import AFTER loading instrument id model

# Set up MyCoBot 280
HOST = "10.42.0.1"
GET_COORDS_PORT = 5006
MOVE_COORDS_PORT = 5005
MOVE_GRIPPER_PORT = 5007
home = [90, 0, 0, -90, 0, -45]

# Set up Mediapipe hand tracking
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)

# Set up depth camera
# TODO: open as a window for Design Day
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
profile = pipeline.start(config)

#inst_img = get_camera_img(pipeline)

c:\Users\maxis\anaconda3\envs\dt12\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
Using cache found in C:\Users\maxis/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2025-4-17 Python-3.9.21 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)

Fusing layers... 
Model summary: 267 layers, 46167513 parameters, 0 gradients, 107.8 GFLOPs
Adding AutoShape... 


Voice activation set up - ONLY RUN THIS IF YOU WANT TO SET A CUSTOM VOICE

In [2]:
choose_tts_voice()


Available English TTS Voices:
[0] Microsoft David Desktop - English (United States) (HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\TTS_MS_EN-US_DAVID_11.0)
[1] Microsoft Zira Desktop - English (United States) (HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\TTS_MS_EN-US_ZIRA_11.0)
Selected voice: Microsoft David Desktop - English (United States)



Main Script

In [3]:
while True:
    home = [90, 0, 0, -90, 0, -45]
    home[3] = home[3] + random.randint(0, 5)
    
    send_coords(home, HOST, MOVE_COORDS_PORT, 1) # send robot to home
    time.sleep(2) # TODO: better way to wait for arm to be stable before getting frame of tray
    inst_img = get_camera_img(pipeline) # get png of tray to run instrument id

    #command = get_voice_command(porcupine, cobra, recorder) # get voice command
    #inst = get_instrument_name(command) # transcribe voice command and get name of instrument
    #instruments = get_instrument_name()
    #inst = instruments[0][0] ### HARD CODED TO JUST TAKE THE FIRST INSTRUMENT IN THE LIST
    inst = 'scissors'
    x_mid, y_mid = identify_instrument(inst_model, inst_img, inst) # get 2d coords of instrument

    ## Get color and depth frame
    frames = pipeline.poll_for_frames()
    if not frames:
        continue

    depth_frame = frames.get_depth_frame()
    color_frame = frames.get_color_frame()

    # Convert yolo instrument coords to 3D coords
    inst_coords = get_inst_coords(color_frame, depth_frame, x_mid, y_mid)
    print(inst_coords)

    # target_coords = inst_coords + [home[3], home[4], home[5]]
    coords = get_coords(HOST, GET_COORDS_PORT)
    if coords is None:
            print("Robot did not return coordinates")
            continue
    
    curr_coords = list(coords)
    target_coords = transform_camera_to_robot(inst_coords, curr_coords[:3], curr_coords[3:])
    target_coords = list(target_coords) + curr_coords[3:]
    print(target_coords)

    pickSequence(target_coords, HOST, MOVE_COORDS_PORT, MOVE_GRIPPER_PORT)
    time.sleep(5)

Response from robot: 


C:\Users\maxis/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


No instrument found
[          0           0           0]
[83.06780435855866, 128.8000030517578, 502.20001220703125, -115.66999816894531, -42.20000076293945, 17.989999771118164]
Gripper command sent: 100 100
Response from robot: 
Gripper command sent: 0 100
Response from robot: 


C:\Users\maxis/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


No instrument found
[          0           0           0]
[83.06780435855866, 134.0999984741211, 396.20001220703125, 178.00999450683594, 2.359999895095825, 44.95000076293945]
Gripper command sent: 100 100
Response from robot: 
Gripper command sent: 0 100
Response from robot: 


...

C:\Users\maxis/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


No instrument found
[          0           0           0]
[81.80006382072068, 139.4000015258789, 404.0, -176.85000610351562, -1.0, 46.150001525878906]
Gripper command sent: 100 100
Response from robot: 
Gripper command sent: 0 100
Response from robot: 


C:\Users\maxis/.cache\torch\hub\ultralytics_yolov5_master\models\common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Box center: (250, 96) | Confidence: 0.93 | Class: Surgical Scissors Sharp/Sharp
[     31.764     -98.594       271.2]
[27.05120133223923, 245.09646060900982, 138.0205787113361, -177.77000427246094, -1.8700000047683716, 45.04999923706055]
Gripper command sent: 100 100
Response from robot: 


KeyboardInterrupt: 